In [1]:
#协同过滤的推荐引擎
#比较用户或者物品之间的相似度，进行推荐
#note:用户的数量很多时，我们倾向于，“基于物品相似度的计算方法”

#物品之间相似度的计算，不是简单的，利用物品的属性进行距离计算，而是
#利用用户对他们的意见进行相似度计算————根据“评价矩阵”（用户 i 对物品 j 的评分矩阵）


#首先定义几个相似度计算函数

import numpy as np
#欧式距离计算相似度
#这里，我们希望，相似度在0-1之间。所以用公式：相似度 = 1/(1+相似度)
def euclidSimilarity(A,B):
    #A,B都是列向量,表明是基于物品的相似度
    dis = np.linalg.norm(A-B)
    return 1.0 / (1.0 + dis) #一定要用1.0

#皮尔逊相似系数
def pearsonSimilarity(A,B):
    if len(A) < 3:
        return 1.0 #检查是否含有三个及以上的点，否则两个向量完全相关
    return 0.5 + 0.5 * np.corrcoef(A,B,rowvar=0)[0,1] #规范到0-1

#余弦相似度
def cosSimilarity(A,B):
    num = float(A.T.dot(B))
    denom = np.linalg.norm(A) * np.linalg.norm(B)
    return 0.5 + 0.5 * (num / denom) #规范到0-1


In [2]:
#构造一个矩阵测试一下

#行表示人，列表示物品:所以矩阵的列表示：基于物品的相似度计算
a = np.array([[1,1,1,0,0],
              [2,2,2,0,0],
              [1,1,1,0,0],
              [5,5,5,0,0],
              [1,1,0,2,2],
              [0,0,0,3,3],
              [0,0,0,1,1]])


S1 = euclidSimilarity(a[:,0],a[:,4])
print S1

S2 = pearsonSimilarity(a[:,0],a[:,4])
print S2

S3 = cosSimilarity(a[:,0],a[:,4])
print S3



0.1336766024
0.237686194076
0.547245559126


In [3]:
#根据相似度函数计算评分
#uersID 表示需要推荐的用户index,  itemID 表示 此用户没有评级过的物品index
def score(data,userID,itemID,simiFunction):
    n = data.shape[1] #物品的数量
    simTotal = 0.0 #相似度的总和
    rateSimTotal = 0.0 #评级的总和
    #对每一个物品进行遍历，
    for j in xrange(n):
        userRating = data[userID,j] #用户已经评级的物品分数
        similarity = 0
        if userRating==0:
            continue #分数为0，表示没有评过级,所以j表示评级的物品index
        #寻找某一个物品J，跟物品itemID,他们都被评级的下标,只要下标，所以有一个[0]
        bothRateIndex = np.nonzero(np.logical_and(data[:,itemID]>0,data[:,j]>0))[0]
        if len(bothRateIndex)==0:
            similarity = 0
        else:
            #基于这些都评级的数字，进行两个物体的相似度计算
            similarity = simiFunction(data[bothRateIndex,itemID],data[bothRateIndex,j])
        simTotal += similarity
        rateSimTotal += similarity * userRating  #itemID的评级 += 与j的相似度 * j的评级 (相当于加权平均)
    if simTotal == 0:
        return 0
    else:
        return rateSimTotal / simTotal #(评级在0-5之间) 
        #相当于加权平均——average_score = score * weight / sum(weight)
    
def recommend(data,userID,favorite = 3,simiFunction = cosSimilarity):
    unratedItems = np.nonzero(data[userID,:] == 0)[0]  #没有评级的index
    if len(unratedItems) == 0:
        return "you have rated everything"
    itemScores = []
    for itemID in unratedItems:
        itemScore = score(data,userID,itemID,simiFunction) #返回评估的分数
        itemScores.append((itemID,itemScore)) 
    return sorted(itemScores,key=lambda x:x[1],reverse=True)[:favorite]


In [4]:
#设置一个矩阵
data = np.array([[4,4,0,2,2],
                 [4,0,0,3,3],
                 [4,0,0,1,1],
                 [1,1,1,2,0],
                 [2,2,2,0,0],
                 [1,1,1,0,0],
                 [5,5,5,0,0]])

recommendation = recommend(data,2)
print recommendation

[(2, 2.5), (1, 2.0243290220056256)]


In [5]:
#test一下numpy的函数
data = np.array([[4,4,0,2,2],
                 [4,0,0,3,3],
                 [4,0,0,1,1],
                 [1,1,1,2,0],
                 [2,2,2,0,0],
                 [1,1,1,0,0],
                 [5,5,5,0,0]])
print np.nonzero(np.logical_and(data[:,1]>0,data[:,2]>0))[0]

print np.nonzero(data[2,:] == 0)[0]

print data[[1,2],0]

[3 4 5 6]
[1 2]
[4 4]
